In [ ]:
param_market = 'US'

In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./item.xlsx',
                       header=0,
                       dtype=str).fillna('')

dict_item = {}
list_job = []
for i in range(len(input_)):
    if input_.loc[i, 'Vehicle_Page'] != '':
        vehicle_page = int(input_.loc[i, 'Vehicle_Page'])
        dict_item[input_.loc[i, 'Item_Number']] = {'vehicle_page': vehicle_page,
                                                   'crawler': 0,
                                                   'dict_vehicle': {},
                                                   'vehicle_1': '',
                                                   'vehicle_2': ''}
        for j in range(vehicle_page):
            list_job.append((i, j))

print('总数量：' + str(len(list_job)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(list_job)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(10):
            try:
                i, j = list_job[a]

                data = '{"scopedContext":{"catalogDetails":{"itemId":"' + input_.loc[i, 'Item_Number'] + '","categoryId":"' + input_.loc[i, 'Category_Id'] + '","marketplaceId":"' + input_.loc[i, 'Market_Id'] + '"}}}'

                ua = UA.get_User_Agent_Requests()
                ua['Content-Type'] = 'application/json'

                if param_market == 'US':
                    url = 'https://www.ebay.com/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=US&_stpos=91710&orig_cvip=true'
                elif param_market == 'DE':
                    url = 'https://www.ebay.de/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=DE&_stpos=10115&orig_cvip=true'
                # elif param_market == 'AU':
                #     pass

                resp = requests.post(url,
                                     data=data,
                                     headers=ua,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                list_header = json_data['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']
                
                # = = = = = = = = = = = = = = =
                
                if param_market == 'US' or param_market == 'AU':
                    name_make = 'Make'
                    name_model = 'Model'
                    name_engine = 'Engine'
                    name_year = 'Year'
                elif param_market == 'DE':
                    name_make = 'Marke'
                    name_model = 'Modell'
                    name_engine = 'Motor'
                    name_year = 'Baujahr'

                for k in range(len(list_header)):
                    if name_make in list_header[k]['textSpans'][0]['text']:
                        col_make = k
                    elif name_model in list_header[k]['textSpans'][0]['text']:
                        col_model = k
                    elif name_engine in list_header[k]['textSpans'][0]['text']:
                        col_engine = k
                    elif name_year in list_header[k]['textSpans'][0]['text']:
                        col_year = k
                        
                # = = = = = = = = = = = = = = =
                
                list_row = json_data['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']

                # = = = = = = = = = = = = = = =
                
                for row in list_row:
                    make = row['cells'][col_make]['textSpans'][0]['text'].strip()
                    model = row['cells'][col_model]['textSpans'][0]['text'].strip()
                    engine = row['cells'][col_engine]['textSpans'][0]['text'].strip()
                    if param_market == 'US' or param_market == 'AU':
                        list_year = [int(row['cells'][col_year]['textSpans'][0]['text'].strip())]
                    elif param_market == 'DE':
                        list_year = [int(year.split('/').strip()) for year in row['cells'][col_year]['textSpans'][0]['text'].split('-')]

                    if make not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make] = {model: {engine: list_year}}
                    else:
                        if model not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model] = {engine: list_year}
                        else:
                            if engine not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] = list_year
                            else:
                                dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] += list_year

                    dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] = list(set(dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine]))
                    dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine].sort()

                dict_item[input_.loc[i, 'Item_Number']]['crawler'] += 1

                # = = = = = = = = = = = = = = =
                
                if dict_item[input_.loc[i, 'Item_Number']]['vehicle_page'] == dict_item[input_.loc[i, 'Item_Number']]['crawler']:
                    list_vehicle = []
                    for make in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        for model in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            list_year = []
                            for engine in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                list_year += dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine]
                            list_year = list(set(list_year))
                            list_year.sort()

                            year_min = list_year[0]
                            year_max = list_year[-1]
                            if year_min == year_max:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                            else:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                    list_vehicle.sort()

                    dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'] = '\n'.join(list_vehicle)
                    
                    # = = = = = = = = = = = = = = =
                    
                    list_vehicle = []
                    for make in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        for model in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            for engine in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                year_min = dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine][0]
                                year_max = dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine][-1]
                                if year_min == year_max:
                                    if engine == '':
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                                    else:
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
                                else:
                                    if engine == '':
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                                    else:
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

                    list_vehicle.sort()

                    dict_item[input_.loc[i, 'Item_Number']]['vehicle_2'] = '\n'.join(list_vehicle)
                    
                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                                             'Vehicle_1': dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'],
                                             'Vehicle_2': dict_item[input_.loc[i, 'Item_Number']]['vehicle_2']}])

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'

                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                                     'Vehicle_Page': input_.loc[i, 'Vehicle_Page'],
                                     'Category_Id': input_.loc[i, 'Category_Id'],
                                     'Market_Id': input_.loc[i, 'Market_Id']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[i, 'Item_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Item_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Item_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：368

[ok] - 284240562623
[尝试次数：1] - [剩余数量：348] - [当前时间：10:08:24]

[ok] - 284238171734
[尝试次数：1] - [剩余数量：347] - [当前时间：10:08:24]

[ok] - 284238171734
[尝试次数：1] - [剩余数量：346] - [当前时间：10:08:24]

[ok] - 284238155559
[尝试次数：1] - [剩余数量：345] - [当前时间：10:08:24]

[ok] - 284240322618
[尝试次数：1] - [剩余数量：344] - [当前时间：10:08:24]

[ok] - 303939114015
[尝试次数：1] - [剩余数量：343] - [当前时间：10:08:24]

[ok] - 303939114015
[尝试次数：1] - [剩余数量：342] - [当前时间：10:08:24]

[ok] - 284240396275
[尝试次数：1] - [剩余数量：341] - [当前时间：10:08:24]

[ok] - 284238155559
[尝试次数：1] - [剩余数量：340] - [当前时间：10:08:24]

[ok] - 303942818317
[尝试次数：1] - [剩余数量：339] - [当前时间：10:08:24]

[ok] - 284240322618
[尝试次数：1] - [剩余数量：338] - [当前时间：10:08:24]

[ok] - 284240396275
[尝试次数：1] - [剩余数量：337] - [当前时间：10:08:24]

[ok] - 284238155559
[尝试次数：1] - [剩余数量：336] - [当前时间：10:08:24]

[ok] - 303939114015
[尝试次数：1] - [剩余数量：335] - [当前时间：10:08:24]

[ok] - 284240322618
[尝试次数：1] - [剩余数量：334] - [当前时间：10:08:24]

[ok] - 303941511458
[尝试次数：1] - [剩余数量：333] - [当前时间：10:08:24]

[ok] - 30394151

[error] - 283603574410
[尝试次数：10] - [剩余数量：215] - [当前时间：10:08:45]

[ok] - 274126414393
[尝试次数：10] - [剩余数量：214] - [当前时间：10:08:45]

[ok] - 273142831802
[尝试次数：4] - [剩余数量：213] - [当前时间：10:08:45]

[error] - 273142816010
[尝试次数：10] - [剩余数量：212] - [当前时间：10:08:45]

[ok] - 303275740146
[尝试次数：2] - [剩余数量：211] - [当前时间：10:08:45]

[ok] - 303274791774
[尝试次数：4] - [剩余数量：210] - [当前时间：10:08:46]

[ok] - 303275740180
[尝试次数：4] - [剩余数量：209] - [当前时间：10:08:46]

[error] - 273142852903
[尝试次数：10] - [剩余数量：208] - [当前时间：10:08:46]

[ok] - 303275740180
[尝试次数：4] - [剩余数量：207] - [当前时间：10:08:46]

[error] - 303274791774
[尝试次数：10] - [剩余数量：206] - [当前时间：10:08:47]

[ok] - 273996458624
[尝试次数：2] - [剩余数量：205] - [当前时间：10:08:47]

[ok] - 273142852903
[尝试次数：6] - [剩余数量：204] - [当前时间：10:08:47]

[error] - 303275740146
[尝试次数：10] - [剩余数量：203] - [当前时间：10:08:47]

[ok] - 273996463185
[尝试次数：3] - [剩余数量：202] - [当前时间：10:08:47]

[error] - 303275740146
[尝试次数：10] - [剩余数量：201] - [当前时间：10:08:47]

[ok] - 273142852903
[尝试次数：10] - [剩余数量：200] - [当前时间：10:08:47]

[error] - 283590051014
[尝试次数：10] - [剩余数量：83] - [当前时间：10:09:10]

[ok] - 274731124271
[尝试次数：1] - [剩余数量：82] - [当前时间：10:09:10]

[ok] - 273977168086
[尝试次数：8] - [剩余数量：81] - [当前时间：10:09:10]

[error] - 283590051014
[尝试次数：10] - [剩余数量：80] - [当前时间：10:09:11]

[ok] - 284230244245
[尝试次数：1] - [剩余数量：79] - [当前时间：10:09:11]

[ok] - 284230242670
[尝试次数：5] - [剩余数量：78] - [当前时间：10:09:11]

[error] - 273977219860
[尝试次数：10] - [剩余数量：77] - [当前时间：10:09:11]

[error] - 274731124271
[尝试次数：10] - [剩余数量：76] - [当前时间：10:09:11]

[ok] - 274731180252
[尝试次数：4] - [剩余数量：75] - [当前时间：10:09:11]

[error] - 283590051014
[尝试次数：10] - [剩余数量：74] - [当前时间：10:09:12]

[ok] - 273977067188
[尝试次数：2] - [剩余数量：73] - [当前时间：10:09:12]

[ok] - 274731124271
[尝试次数：10] - [剩余数量：72] - [当前时间：10:09:12]

[error] - 284230242670
[尝试次数：10] - [剩余数量：71] - [当前时间：10:09:12]

[error] - 284230242670
[尝试次数：10] - [剩余数量：70] - [当前时间：10:09:12]

[ok] - 273977168086
[尝试次数：7] - [剩余数量：69] - [当前时间：10:09:13]

[ok] - 274731180252
[尝试次数：6] - [剩余数量：68] - [当前时间：10:09:13]

[error] - 2